In [115]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  0


In [116]:
!module load tensorflow/2.0.0 

Loading tensorflow/2.0.0
  Loading requirement: intel-mkl/2019.3.199 python3/3.7.4 cuda/10.1
    cudnn/7.6.5-cuda10.1 nccl/2.5.6-1+cuda10.1 openmpi/4.0.1
>

In [117]:
import importlib
import ensemble
import keras
importlib.reload(ensemble)
import numpy as np

In [118]:
# Use the feature vectors Jack generated from one of the fusion model's intermediary layers

def get_arrays(partition):
    import numpy as np
    features = np.load(f"/g/data/x77/jm0124/feature_vectors/feature-array-latest-{partition}.npy")
    import json
    with open(f"/g/data/x77/jm0124/feature_vectors/{partition}_feature_labels.json", "r") as f:
        labels_dict = json.load(f)
    ids = list(labels_dict.keys())
    labels = np.array([[y for y in labels_dict[id_]['label'][:2]] for id_ in ids])
    return features, labels

# each row is the feature vector for a time interval
train_feature_vectors, train_label_vectors = get_arrays("train")
print(train_feature_vectors.shape, train_label_vectors.shape)

valid_feature_vectors, valid_label_vectors = get_arrays("val")
print(valid_feature_vectors.shape, valid_label_vectors.shape)

(166686, 128) (166686, 2, 2)
(21122, 128) (21122, 2, 2)


In [119]:
print(train_label_vectors)

[[[129.09   129.047 ]
  [ 25.9511  28.0132]]

 [[129.5    128.962 ]
  [ 26.3     28.5154]]

 [[129.393  128.626 ]
  [ 26.4315  29.0259]]

 ...

 [[108.     107.2   ]
  [ 20.5     21.4   ]]

 [[107.698  107.11  ]
  [ 20.8949  21.4375]]

 [[107.5    107.    ]
  [ 21.2     21.5   ]]]


In [120]:
# train all of the models in model_classes on the training data
def train(model_class, **kwargs):
    model_dir = kwargs.get('model_dir', None)
    
    model = model_class()
    print("\n\nTraining model " + model.NAME)
    if model_dir != None:
        model = keras.models.load_model(model_dir)
    model.train(train_feature_vectors, valid_feature_vectors, train_label_vectors, valid_label_vectors, verbose=True)
    print("Saving model " + model.NAME + " which had validation mean km error of " + str(model.mean_km))
    model.save(model.NAME + "-real-data")
# for model_class in ensemble.model_classes: train_model(model_class)

In [121]:
from ensemble import haversine_loss_tf

label = tf.convert_to_tensor(train_label_vectors[0:3], dtype='float32')
print(label)
feature = tf.convert_to_tensor(train_feature_vectors[0:3], dtype='float32')
prediction = tf.convert_to_tensor([[-3, 2.5],
                                    [0.2,0.2],
                                    [0.2,0.2]])
# print(f"Label: {label}")
# print(f"Prediction: {prediction}")


# y_true = label[1,:]
# print(f"True location: {y_true}")

print(f"Label: {label}")
print(f"Prediction: {prediction}")
print(haversine_loss_tf(label,prediction))
# haversine_loss_tf(

tf.Tensor(
[[[129.09   129.047 ]
  [ 25.9511  28.0132]]

 [[129.5    128.962 ]
  [ 26.3     28.5154]]

 [[129.393  128.626 ]
  [ 26.4315  29.0259]]], shape=(3, 2, 2), dtype=float32)
Label: [[[129.09   129.047 ]
  [ 25.9511  28.0132]]

 [[129.5    128.962 ]
  [ 26.3     28.5154]]

 [[129.393  128.626 ]
  [ 26.4315  29.0259]]]
Prediction: [[-3.   2.5]
 [ 0.2  0.2]
 [ 0.2  0.2]]
tf.Tensor([293.74368 235.62312 282.71185], shape=(3,), dtype=float32)


In [128]:
# sydney = (-33.8688, 151.2093)
# canberra = (-35.2802, 149.1310)

target_tensor = np.array([[
    [0, 151.2093],
    [0, -33.8688],
                            ],
    [
    [0, 153.0260],
    [0, -27.4705],
                            ]])

pred_tensor = np.array([
    [149.1310, -35.2802],
    [145.7710, -16.9203]
                        ])

print(target_tensor)
print(pred_tensor)

print(haversine_loss_tf(tf.convert_to_tensor(target_tensor),tf.convert_to_tensor(pred_tensor)))

[[[  0.     151.2093]
  [  0.     -33.8688]]

 [[  0.     153.026 ]
  [  0.     -27.4705]]]
[[149.131  -35.2802]
 [145.771  -16.9203]]
tf.Tensor([ 246.64132337 1389.84865687], shape=(2,), dtype=float64)


In [133]:
import ensemble
importlib.reload(ensemble)

<module 'ensemble' from '/home/156/jm0124/dl-cyclones/ensemble/ensemble.py'>

In [ ]:
train(ensemble.ANNModel)



Training model ANN
Model: "sequential_37"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_87 (Dense)            (None, 2048)              264192    
                                                                 
 batch_normalization_48 (Bat  (None, 2048)             8192      
 chNormalization)                                                
                                                                 
 dense_88 (Dense)            (None, 2)                 4098      
                                                                 
Total params: 276,482
Trainable params: 272,386
Non-trainable params: 4,096
_________________________________________________________________
Epoch 1/100
651/651 - 5s - loss: 331.5630 - val_loss: 320.2931 - 5s/epoch - 8ms/step
Epoch 2/100
651/651 - 4s - loss: 299.0083 - val_loss: 304.8735 - 4s/epoch - 7ms/step
Epoch 3/100


In [125]:
train(ensemble.AdaBoostModel)



Training model ADA
Fitting 5 folds for each of 1 candidates, totalling 5 fits


/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.01/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
5 fits failed out of a total of 5.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.01/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.01/lib/python3.9/site-packages/sklearn/ensemble/_weight_boosting.py", line 1065, in fit
    return super().fit(X, y, sample_weight)
  File "/g/data/hh5/public/apps/minico

ValueError: y should be a 1d array, got an array of shape (166686, 2) instead.

In [126]:
train(ensemble.KNNModel)



Training model KNN
Fitting 5 folds for each of 1 candidates, totalling 5 fits


/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.01/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
5 fits failed out of a total of 5.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.01/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.01/lib/python3.9/site-packages/sklearn/neighbors/_regression.py", line 213, in fit
    return self._fit(X, y)
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-22

ValueError: Found array with dim 3. Estimator expected <= 2.

In [ ]:
train(ensemble.RandomForestModel)



Training model RF
Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


In [127]:
train(ensemble.XGBModel)



Training model XGB


ValueError: Invalid shape: (166686, 2, 2) for label

In [153]:
# Save/load models, check they work after loading from disk
models = [model_class.load(model_class.NAME + "-real-data") for model_class in ensemble.model_classes]

In [154]:
for model in models:
    print(f"{model.NAME:<8} val loss: {model.mean_km:.2f}")

Min possible loss: 145.56
ANN      val loss: 186.40
ADA      val loss: 182.16
KNN      val loss: 184.31
RF       val loss: 186.20
XGB      val loss: 203.14


In [164]:
# Ensemble these models together and see how the ensemble loss compares
preds = np.array([model.predict(valid_feature_vectors) for model in models])
print([model.NAME for model in models])
print([preds[i].shape for i in range(len(preds))])
preds.shape

['ANN', 'ADA', 'KNN', 'RF', 'XGB']
[(50, 2), (50, 2), (50, 2), (50, 2), (50, 2)]


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    0.0s finished


(5, 50, 2)

In [165]:
ensemble_preds = np.swapaxes(np.swapaxes(preds, 0, 2), 0, 1).mean(axis=2)
haversine_loss(ensemble_preds, valid_label_vectors).mean()

180.51402616749732